## An introduction to xbowflow workflows

In this notebook you will see how a simple MD simulation job can be converted from its normal command-line form into a Python function using tools in *Xbowflow*.

Then you will see how it's easy to chain jobs together to create a workflow.

The notebook assumes you have a basic knowledge of *Gromacs*, and that *Gromacs* is installed on the computer you are running this notebook on.

Some knowledge of the Python package *MDTraj* may also help, but is not obligatory.

----

### Part 1: running jobs the conventional way
Have a look at the contents of this directory:

In [ ]:
!ls

You should see:

    Xbowflow workflows 101.ipynb : This notebook
    bpti.gro                     : Coordinates for BPTI in Gromacs .gro format
    bpti.top                     : Gromacs topology file for BPTI
    em.mdp                       : A Gromacs .mdp file defining an energy minimisation job
    nvt.mdp                      : a Gromacs .mdp file defining a short NVT MD simulation
    
Let's begin by running the energy minimisation job interactively in the conventional way. 

First we run grompp:

In [ ]:
!gmx grompp -f em.mdp -c bpti.gro -p bpti.top -o bpti-em.tpr

Assuming everything there went as expected, now we can run the energy minimisation itself:

In [ ]:
! gmx mdrun -s bpti-em.tpr -c bpti-em.gro -g bpti-em.log -o bpti-em.trr -e bpti-em.edr

Assuming the job completed without errors, you should see the output files in the current directory:

In [ ]:
!ls

Have a look at the log file:

In [ ]:
!cat bpti-em.log

### Part 2: Turning this into Python

OK. Now you will see how you can turn the energy minimisation job from something you run on the command line (in this situation, within a Jupyter notebook, by using the "!" special command) into a pure Python function.

The function will take a .tpr file as the input, and return the .gro and .log files when the job completes. For now, you can assume you are not that bothered about what's in the .edr and .trr files.

So your aim is something like this:

    grofile, logfile = mdrun(tprfile)
    
---

Begin by importing the *xflowlib* module:

In [ ]:
from xbowflow import xflowlib

Now you create the function, which in xflowlib is called a **Kernel**:

In [ ]:
md = xflowlib.SubprocessKernel('gmx mdrun -s {x.tpr} -c {x.gro} -g {x.log} -e {x.edr} -o {x.trr}')

You can see that creating the function (or "kernel") involves providing a "template" for the command you want to run. The names of the files in the template are completely up to you (e.g. you could use "system.tpr", etc. instead of "x.tpr") - but in general make sure the filenames have the appropriate extensions.

---

Now you have to tell the kernel what files are inputs, and what are outputs. To do this you pass *lists* of strings that correspond to the filenames in the template above. 

**NB:** the order of the strings in the inputs list defines the order that input variables will be passed to the kernel, and the order of the strings in the output list defines the order that the outputs from the function will appear in:

In [ ]:
# Give the kernel the signature: grofile, logfile = mdrun.run(tprfile)
md.set_inputs(['x.tpr'])
md.set_outputs(['x.gro', 'x.log'])

And that's about it - your new function is ready for use.

However, your data is not quite ready. Xbowflow is designed to work with distributed computing facilities that may not share a common file system. So before you can use the function, you need to get the input .tpr file into a suitable globally-accessible variable:

In [ ]:
xflowlib.set_filehandler('memory')
tprfile = xflowlib.load('bpti-em.tpr')

Now you can run the function, by caling its run() method:

In [ ]:
grofile, logfile = md.run(tprfile)

Before you can look at the results, you need to convert the globally-accessible output variables back into files:

In [ ]:
logfile.save('test.log')
grofile.save('test.gro')

In [ ]:
!cat test.log

In [ ]:
!cat test.gro

What happened to the information that gets written to the screen when you run the job via the command line? It's captured in the STDOUT attribute of the kernel:

In [ ]:
print(md.STDOUT)

### Part 3: A workflow

Let's make a workflow that runs a grompp job, then immediately the md (or energy minimisation) job.

You already have a kernel that can run *mdrun*, but you need to build one to run *grompp*:

In [ ]:
# Build a kernel with the signature: tprfile = grompp.run(mdpfile, grofile, topfile):
grompp = xflowlib.SubprocessKernel('gmx grompp -f {x.mdp} -c {x.gro} -p {x.top} -o {x.tpr} -maxwarn 1')
grompp.set_inputs(['x.mdp', 'x.gro', 'x.top'])
grompp.set_outputs(['x.tpr'])

See if it works:

In [ ]:
# Create variables from the required input files:
emfile = xflowlib.load('em.mdp')
start_crds = xflowlib.load('bpti.gro')
topfile = xflowlib.load('bpti.top')
# Run the job:
em_tprfile = grompp.run(emfile, start_crds, topfile)

The output from this kernel should be ready for use in the mdrun kernel - let's see:

In [ ]:
# For clarity, re-run the grompp job:
em_tprfile = grompp.run(emfile, start_crds, topfile)
# Now the energy minimisation:
em_crds, em_logfile = md.run(em_tprfile)

em_logfile.save('em.log')

In [ ]:
!cat em.log

### Part 4: Exercise - a bigger workflow

Your turn - add the second simulation stage - the NVT MD - into your workflow.

Hints:
    1. You don't need to make any new kernels - re-use the ones you have.
    2. Don't forget that you will need to create a variable from the file nvt.mdp

In [ ]:
# A workflow that runs an energy minimisation and then an NVT MD simulation
# For clarity, start at the beginning:
em_tprfile = grompp.run(emfile, start_crds, topfile)
em_crds, em_logfile = md.run(em_tprfile)
# Add your code below:
nvtfile = xflowlib.load('nvt.mdp')
nvt_tprfile = grompp.run(nvtfile, em_crds, topfile)
nvt_crds, nvt_logfile = md.run(nvt_tprfile)


### Part 5: A better workflow

Let's improve the workflow. Firstly, it would be nice if the NVT simulation job also returned the trajectory file. You don't want this for the EM job, so what that means is that you need to make a second mdrun-type kernel. Here it is:

In [ ]:
md_with_traj = md.copy()
md_with_traj.set_outputs(['x.gro', 'x.log', 'x.trr'])

The copy() convenience method saves you having to rewrite the kernel from scratch, if  it's just a tweak on an existing one.

Next, notice that both grompp jobs in the workflow above take the same topology file as an argument - in effect, it's a constant. In such cases, you can define it as such at the time you create the kernel, and then you don't have to include it in the list of arguments when you call it:

In [ ]:
grompp.set_constant('x.top', topfile)
# Now the new improved workflow:
em_tprfile = grompp.run(emfile, start_crds)
em_crds, em_logfile = md.run(em_tprfile)
nvt_tprfile = grompp.run(nvtfile, em_crds)
nvt_crds, nvt_logfile, nvt_traj = md_with_traj.run(nvt_tprfile)

### Part 6: interfacing with more Python

At this stage you may be thinking "OK - but nothing here I couldn't do with a bash script". The power of the workflow comes when you interface your new pythonized-MD functions with other Python tools.

Let's make use of the *MDTraj* package for analysis of MD trajectory data. You will use it to calculate the RMSD of the trajectory frames from the starting structure.

If you are not yet familiar with MDTraj don't worry - what's below should be more or less self-explanatory.

The MDTraj load() method expects conventional  *filenames* as arguments. For this, you can use the as_file() method of a variable created by xbowflow.load():

In [ ]:
import mdtraj as mdt
traj = mdt.load(nvt_traj.as_file(), top=start_crds.as_file())
print(traj)
# Calculate the rmsd of each frame from the first:
print(mdt.rmsd(traj, traj[0], atom_indices=traj.topology.select('protein')))

Let's make your workflow identify which snapshot from your trajectory has the highest RMSD from the starting structure, and then energy minimise that:

In [ ]:
import numpy as np
rmsdlist = mdt.rmsd(traj, traj[0], atom_indices=traj.topology.select('protein'))
i = np.argmax(rmsdlist)
print('Energy minimising snapshot {}'.format(i))
em2_tprfile = grompp.run(emfile, traj[i])
em2_crds, em2_logfile = md.run(em2_tprfile)
em2_crds.save('max_rmsd.gro')

### Part 7: Final exercise

Create a Python function that in effect does all the above: takes a set of starting coordinates, a topology file, and two .mdp files (one for an energy minimisation, one for an MD run), runs the workflow and then returns the energy-minimised structure of the snapshot with the highest RMSD from the starting structure. The function should do everything, including creating the required kernels:

In [ ]:
def my_workflow(crd_filename, top_filename, em_mdp_filename, md_mdp_filename):
    # Over to you!
    # Load data:
    start_crds = xflowlib.load(crd_filename)
    topfile = xflowlib.load(top_filename)
    emfile = xflowlib.load(em_mdp_filename)
    mdfile = xflowlib.load(md_mdp_filename)
    
    # Create kernels:
    grompp = xflowlib.SubprocessKernel('gmx grompp -f {x.mdp} -c {x.gro} -p {x.top} -o {x.tpr} -maxwarn 1')
    grompp.set_inputs(['x.mdp', 'x.gro'])
    grompp.set_constant('x.top', topfile)
    grompp.set_outputs(['x.tpr'])
    
    md = xflowlib.SubprocessKernel('gmx mdrun -s {x.tpr} -c {x.gro} -g {x.log} -e {x.edr} -o {x.trr}')
    md.set_inputs(['x.tpr'])
    md.set_outputs(['x.gro', 'x.log'])
    
    md_with_traj = md.copy()
    md_with_traj.set_outputs(['x.gro', 'x.log', 'x.trr'])
    
    # Run workflow:
    em_crds, em_logfile = md.run(grompp.run(emfile, start_crds))
    md_crds, md_logfile, md_traj = md_with_traj.run(grompp.run(mdfile, em_crds))
    traj = mdt.load(md_traj.as_file(), top=start_crds.as_file())
    rmsdlist = mdt.rmsd(traj, traj[0], atom_indices=traj.topology.select('protein'))
    i = np.argmax(rmsdlist)
    print('Energy minimising snapshot {}'.format(i))
    em2_crds, em2_logfile = md.run(grompp.run(emfile, traj[i]))
    
    # Return final structure:
    return em2_crds

# Test the workflow:
final_crds = my_workflow('bpti.gro', 'bpti.top', 'em.mdp', 'nvt.mdp')
final_crds.save('final_coordinates.gro')